<a href="https://colab.research.google.com/github/Tharindu021/Context_Based_Spell_Checker/blob/main/SpellChecker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import NLTK is a natural language toolkit and the wordnet is a database it can use for the spell checker and also lingustic events.and from that we can get edit distance method also

In [32]:
import nltk
from nltk.corpus import wordnet
from nltk.metrics.distance import edit_distance
import pandas as pd
import numpy as np
import gensim
from gensim.models import Word2Vec
import string
from nltk.corpus import stopwords
from transformers import BertTokenizer, BertModel

i try to check with in the dataset in the drive.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# so i need to train a model using word2vec to get simillar words to the recomendation words

firstly need to get the dataset to the training the model

In [6]:
#dataset_path = "/content/drive/MyDrive/Dataset/All_train_data.csv"
#dp = pd.read_csv(dataset_path);
#dp['input'].head()
from gensim.models import KeyedVectors
model_path = "/content/drive/MyDrive/Dataset/Models/google_news.kv"
model = KeyedVectors.load(model_path)


then i need to apply preprocess before the model training

In [ ]:
#dp_text = dp.input.apply(gensim.utils.simple_preprocess)
from datasets import list_datasets

# Get the list of available datasets
datasets_list = list_datasets()

# Print the list of datasets
for dataset_name in datasets_list:
    print(dataset_name)


after the preprocess we can tune the word2vec for our needed

In [ ]:
model = gensim.models.Word2Vec(
    vector_size = 150,
    window = 10,
    workers = 6,
    min_count = 4
)

then we can train the model using the dataset the we can finalize the trainning

In [ ]:
model.build_vocab(dp_text,progress_per=100)
model.train(dp_text, total_examples=model.corpus_count , epochs = 5)

(998274, 1410485)

In [10]:
model["apple"]

array([-0.06445312, -0.16015625, -0.01208496,  0.13476562, -0.22949219,
        0.16210938,  0.3046875 , -0.1796875 , -0.12109375,  0.25390625,
       -0.01428223, -0.06396484, -0.08056641, -0.05688477, -0.19628906,
        0.2890625 , -0.05151367,  0.14257812, -0.10498047, -0.04736328,
       -0.34765625,  0.35742188,  0.265625  ,  0.00188446, -0.01586914,
        0.00195312, -0.35546875,  0.22167969,  0.05761719,  0.15917969,
        0.08691406, -0.0267334 , -0.04785156,  0.23925781, -0.05981445,
        0.0378418 ,  0.17382812, -0.41796875,  0.2890625 ,  0.32617188,
        0.02429199, -0.01647949, -0.06494141, -0.08886719,  0.07666016,
       -0.15136719,  0.05249023, -0.04199219, -0.05419922,  0.00108337,
       -0.20117188,  0.12304688,  0.09228516,  0.10449219, -0.00408936,
       -0.04199219,  0.01409912, -0.02111816, -0.13476562, -0.24316406,
        0.16015625, -0.06689453, -0.08984375, -0.07177734, -0.00595093,
       -0.00482178, -0.00089264, -0.30664062, -0.0625    ,  0.07

in this we can retrive the data from the dataset to use to the spell checking and then finally returns as the array

In [11]:
path = "/content/drive/MyDrive/Dataset/english Dictionary only words.csv"

df = pd.read_csv(path)
dictionary = set(df['word'].str.lower())
print(df['word'].tail())
#with open(path,'r') as file:
#    words = file.read().split()
#  return set(words)


<ipython-input-11-7a37bfac810e>:3: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


188131    Yupon
188132      Yux
188133     Yvel
188134     Ywar
188135     Ywis
Name: word, dtype: object


punkt is a tokenizer and we need to download that and the wordnet database also then we can preprocessing our project

In [12]:
nltk.download('punkt')
#nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

 i use cosine similarity to measure the similarity between two vectors


In [29]:
def compute_similarity(vector1, vector2):
    return np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))

I am using downloaded pretained model useto the checking the spell checking and also following function is to find most similar_word in given input words to the sentence, Firstly it tokenise the sentence and then check the words are in the model if not it return none and next get the sentence mean base on the word it store in the sentence_mean then it measure the cosine similarity between sentence and relevant word and it label to the word then its store in the simillarities variable then sort it in descending order return the most 5 similar word.

In [35]:
def find_most_similar_word(sentence, input_words, word2vec_model):
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))
    sentence = nltk.word_tokenize(sentence)
    words = [word for word in sentence if word in word2vec_model]
    if not words:
        return None

    sentence_mean = np.mean([word2vec_model[word] for word in words], axis=0)
    input_words = [input_word for input_word in input_words if input_word in word2vec_model]
    similarities = [(label, compute_similarity(sentence_mean, word2vec_model[label])) for label in input_words]

    print(similarities)
    similarities.sort(key=lambda x: x[1], reverse=True)


    return similarities[0:5]

from below code we can get the input sentence and that sentence can tokenize using nltk.word_tokenize then we need to chech that words one by one then if it is not in the dataset then we can print there is no word like user input and show then to the what word are the mispelled and the recomendations using edit distance algorithm

In [34]:
def spell_checker(sentence, dictionary):
      simillar_word = []
      sentence = sentence.translate(str.maketrans('', '', string.punctuation))
      words = nltk.word_tokenize(sentence)
      for word in words:
          if word.lower() not in dictionary:
             simillar_word = recomend_simillar_words(word.lower())
             #print(word,simillar_word,sentence)
             recomended_words = find_most_similar_word(sentence,simillar_word,model)
             print("Recommended words for '{}' are: {}".format(word, recomended_words))

in this code we can get the simmilar words for misspelled words that are input have i choose only below 2 distance. and then recomend to the client only 10 words.

In [15]:
def recomend_simillar_words(words):
  simillar_words = []
  for dict_words in dictionary:
    distance = edit_distance( dict_words , words )
    if distance <= 1:
      simillar_words.append( (dict_words , distance) )
  simillar_words.sort(key=lambda x : x[1])
  return [ word[0] for word in simillar_words[:20]]

in that code part we can input the sentence then it strictly move to the spell_checker code part then it can check whether the word is in the dataset or not

In [37]:

user_input = ""
while user_input.lower() != "exit":
    user_input = input("Please enter the sentence you want or type 'exit' to quit: ")
    if user_input.lower() == "exit":
        print("Exiting...")
    else:
        spell_checker(user_input, dictionary)

Please enter the sentence you want or type 'exit' to quit: i eat aple
[('axle', 0.08812634), ('able', 0.18671234), ('maple', 0.13715714), ('apple', 0.3208351), ('ale', 0.17726763), ('apse', 0.058557026), ('ample', 0.079364814), ('ape', 0.19740877)]
Recommended words for 'aple' are: [('apple', 0.3208351), ('ape', 0.19740877), ('able', 0.18671234), ('ale', 0.17726763), ('maple', 0.13715714)]
Please enter the sentence you want or type 'exit' to quit: i eat aple.
[('axle', 0.08812634), ('able', 0.18671234), ('maple', 0.13715714), ('apple', 0.3208351), ('ale', 0.17726763), ('apse', 0.058557026), ('ample', 0.079364814), ('ape', 0.19740877)]
Recommended words for 'aple' are: [('apple', 0.3208351), ('ape', 0.19740877), ('able', 0.18671234), ('ale', 0.17726763), ('maple', 0.13715714)]
Please enter the sentence you want or type 'exit' to quit: exit
Exiting...
